In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


# Importing Packages

In [1]:
import pandas as pd
import numpy as np
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model

In [2]:
#data_cleaning 
#data_path='/content/drive/My Drive/Colab Notebooks/LanguageTranslation/hin.txt'
data_path=r"E:\DATASETS\LanguageTranslation\hindi_english_data\hin.txt"
lines= pd.read_table(data_path, names=['eng', 'hin'],encoding='utf-8')

In [3]:
lines.drop('hin',inplace=True,axis='columns')

In [4]:
lines.rename(columns={'eng':'hin'},inplace=True)

In [5]:

lines['eng']=lines.index

In [6]:
lines.reset_index(level=0, inplace=True)

In [7]:
lines.drop('index',inplace=True,axis='columns')
lines.head()

,hin,eng
0,वाह!,Wow!
1,बचाओ!,Help!
2,उछलो.,Jump.
3,कूदो.,Jump.
4,छलांग.,Jump.


# Data Preprocessing 

In [8]:
#preprocessing of data
# Lowercase all characters
lines.eng=lines.eng.apply(lambda x: x.lower())
lines.hin=lines.hin.apply(lambda x: x.lower())

# Remove quotes
lines.eng=lines.eng.apply(lambda x: re.sub("'", '', x))
lines.hin=lines.hin.apply(lambda x: re.sub("'", '', x))
exclude = set(string.punctuation) # Set of all special characters

# Remove all the special characters
lines.eng=lines.eng.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines.hin=lines.hin.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

# Remove all numbers from text
remove_digits = str.maketrans('', '', digits)
lines.eng=lines.eng.apply(lambda x: x.translate(remove_digits))
lines.hin = lines.hin.apply(lambda x: re.sub("[२३०८१५७९४६]", "", x))

# Remove extra spaces
lines.eng=lines.eng.apply(lambda x: x.strip())
lines.hin=lines.hin.apply(lambda x: x.strip())
lines.eng=lines.eng.apply(lambda x: re.sub(" +", " ", x))
lines.hin=lines.hin.apply(lambda x: re.sub(" +", " ", x))

# Add start and end tokens to target sequences
lines.hin = lines.hin.apply(lambda x : 'START_ '+ x + ' _END')

In [10]:
lines.head(10)

,hin,eng
0,START_ वाह _END,wow
1,START_ बचाओ _END,help
2,START_ उछलो _END,jump
3,START_ कूदो _END,jump
4,START_ छलांग _END,jump
5,START_ नमस्ते। _END,hello
6,START_ नमस्कार। _END,hello
7,START_ वाहवाह _END,cheers
8,START_ चियर्स _END,cheers
9,START_ समझे कि नहीं _END,got it


# Creating Vocab

In [11]:
#Vocabulary of English
all_eng_words=set()
for eng in lines.eng:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)

# Vocabulary of Hindi 
all_hindi_words=set()
for hin in lines.hin:
    for word in hin.split():
        if word not in all_hindi_words:
            all_hindi_words.add(word)


In [13]:
count=-1
for i in all_eng_words:
    count+=1
    if count < 10:
        print(i)#printing some words from english words set in the vocab

plus
jobs
hardly
brothers
team
too
bombay
bags
whom
kid


In [14]:
count=-1
for i in all_hindi_words:
    count+=1
    if count < 10:
        print(i)#printing some words from hindi words set in the vocab

उद्योग
पकड़ीं
जाएँगे।
कर
चोरी
चहिए
तोला।
गईं
द्वार
चुना


In [15]:
# Max Length of source sequence or we can english sentence instead of source sequence
lenght_list=[]
for l in lines.eng:
    lenght_list.append(len(l.split(' ')))
max_length_src = np.max(lenght_list)
max_length_src

22

In [16]:
# Max Length of target sequence or we can hindi sentence instead of source sequence
lenght_list=[]
for l in lines.hin:
    lenght_list.append(len(l.split(' ')))
max_length_tar = np.max(lenght_list)
max_length_tar

27

In [17]:
input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_hindi_words))
num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_hindi_words)
#max_encoder_seq_length = max([len(txt.split()) for txt in lines.eng])
#max_decoder_seq_length = max([len(txt.split()) for txt in lines.hin])
print('encoder token',num_encoder_tokens, 'decoder token',num_decoder_tokens)
#print('max_encoder_seq_len',max_encoder_seq_length,'max_decoder_seq_len',max_decoder_seq_length)


encoder token 2341 decoder token 2968


In [18]:
num_decoder_tokens += 1 # For zero padding
num_decoder_tokens

2969

In [19]:
num_encoder_tokens+=1

In [20]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

In [21]:
print(input_token_index)
print(target_token_index)

{'a': 1, 'abandoned': 2, 'ability': 3, 'ablaze': 4, 'able': 5, 'about': 6, 'above': 7, 'abroad': 8, 'absence': 9, 'absent': 10, 'absolute': 11, 'absurd': 12, 'abused': 13, 'accepted': 14, 'access': 15, 'accident': 16, 'accidental': 17, 'accompanied': 18, 'accompany': 19, 'according': 20, 'account': 21, 'accountable': 22, 'accused': 23, 'accustomed': 24, 'ache': 25, 'acknowledgement': 26, 'acquaintance': 27, 'acquaintances': 28, 'acquainted': 29, 'across': 30, 'act': 31, 'actions': 32, 'actor': 33, 'actress': 34, 'add': 35, 'adding': 36, 'address': 37, 'admit': 38, 'adopted': 39, 'advantage': 40, 'advice': 41, 'advise': 42, 'advised': 43, 'affected': 44, 'afford': 45, 'afraid': 46, 'africa': 47, 'after': 48, 'afternoon': 49, 'again': 50, 'against': 51, 'age': 52, 'ago': 53, 'agree': 54, 'agreement': 55, 'aids': 56, 'air': 57, 'airport': 58, 'alarm': 59, 'alcohol': 60, 'alike': 61, 'alive': 62, 'all': 63, 'allergic': 64, 'allow': 65, 'allowances': 66, 'allowed': 67, 'almost': 68, 'alone'

In [22]:
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

In [23]:
lines = shuffle(lines)
lines.head(10)

,hin,eng
2496,START_ तुम नसीब वालो हो जिसके इतना प्यार करने ...,you are fortunate to have such loving parents
113,START_ मोटे मत हो जाना। _END,dont get fat
1652,START_ जो अखबार में लिखा है वह सच है। _END,what the newspapers say is true
277,START_ यह सही नहीं है। _END,this isnt right
862,START_ उसकी राष्ट्रीयता क्या है _END,what is his nationality
1673,START_ वह हर सुबह अखबार पढ़ता है। _END,he reads the paper every morning
2729,START_ नई सरकार ने देश को भ्रष्टाचार से मुक्त ...,the new government promised to rid the country...
1682,START_ पता नहीं कैसे पर तुमने कर दिया। _END,i dont know how but you did it
2264,START_ उसने कपड़े उतारे बिना ही पानी में छलाँग...,he jumped into the water clothes and all
1501,START_ कृपया ध्यान दीजिए। _END,give me your attention please


In [24]:
X, y = lines.eng, lines.hin
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1)
X_train.shape, X_test.shape

((2495,), (278,))

In [25]:
len(X)

2773

In [26]:
y_train[:10]

585                        START_ तुम कब लौट कर आओगे _END
198                           START_ सब लोग कैसे हैं _END
77                                START_ तुम कैसे हो _END
350                           START_ तू कहाँ रहता है _END
744             START_ स्कूल साढ़े तीन बजे छूटता है। _END
269                      START_ आपसे मिलकर खुशी हुई। _END
1885       START_ मैं पियानो बहुत अच्छा बजा लेता था। _END
2151    START_ उसने अपनी गाड़ी बिना हिचकिचाहट हे बेच द...
561                          START_ यह मुफ़्त का है। _END
1478                   START_ पत्र किसको लिखा गया था _END
Name: hin, dtype: object

In [27]:
X_train[:10]

585                       when will you return
198                            how is everyone
77                                 how are you
350                          where do you live
744                          school is over at
269                           nice to meet you
1885        i was able to play piano very well
2151    he sold his own car without hesitation
561                       it is free of charge
1478             who was the letter written to
Name: eng, dtype: object

In [91]:
X_train.to_pickle('/content/drive/My Drive/Colab Notebooks/LanguageTranslation/Weights_Mar/X_train.pkl')
X_test.to_pickle('/content/drive/My Drive/Colab Notebooks/LanguageTranslation/Weights_Mar/X_test.pkl')

# Encoder - Decoder data for training

In [53]:
encoder_input_data = np.zeros(
    (len(X), max_length_src), #using all data becaus the dataset is small but if we have bigger dataset we can we use split data as well
    dtype='float32')
decoder_input_data = np.zeros(
    (len(X), max_length_tar),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(X), max_length_tar, num_decoder_tokens), #we can  also pass the pass y_train bcoz we just need len nothing else
    dtype='float32')

   
print("encoder input shape",encoder_input_data.shape,'decoder input shape',decoder_input_data.shape,'decoder out shape',decoder_target_data.shape)
for i, (input_text, target_text) in enumerate(zip(X, y)):
    for t, word in enumerate(input_text.split()):
            encoder_input_data[i, t] = input_token_index[word] # encoder input seq
    for t, word in enumerate(target_text.split()):
            print('target',word)
            if t<len(target_text.split())-1:
                decoder_input_data[i, t] = target_token_index[word] # decoder input seq
            if t>0:
              # decoder target sequence (one hot encoded)
              # does not include the START_ token
              # Offset by one timestep
              decoder_target_data[i, t - 1, target_token_index[word]] = 1


encoder input shape (2773, 22) decoder input shape (2773, 27) decoder out shape (2773, 27, 2969)
target START_
target तुम
target नसीब
target वालो
target हो
target जिसके
target इतना


In [52]:
print(decoder_input_data[:2])
print(y[:2])
target_token_index["हो"]
print(decoder_target_data[:2])

[[1.000e+00 1.219e+03 1.456e+03 2.535e+03 2.943e+03 1.052e+03 2.080e+02
  1.744e+03 4.210e+02 2.534e+03 2.169e+03 2.939e+03 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00]
 [1.000e+00 2.268e+03 2.126e+03 2.943e+03 1.032e+03 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00]]
2496    START_ तुम नसीब वालो हो जिसके इतना प्यार करने ...
113                          START_ मोटे मत हो जाना। _END
Name: hin, dtype: object
[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]


In [195]:
#HERE THE DATASET IS SO SMALL WE CAN USE THIS CODE FOR GENERATING VALIDATING MATRICES FOR INPUT

'''encoder_input_val_data = np.zeros(
    (len(X_test), max_length_src),
    dtype='float32')
decoder_input_val_data = np.zeros(
    (len(X_test), max_length_tar),
    dtype='float32')
decoder_target_val_data = np.zeros(
    (len(X_test), max_length_tar, num_decoder_tokens), #we can  also pass the pass y_train bcoz we just need len nothing else
    dtype='float32')


   
for i, (input_text, target_text) in enumerate(zip(X_test, y_test)):
      for t, word in enumerate(input_text.split()):
          encoder_input_val_data[i, t] = input_token_index[word] # encoder input seq
      for t, word in enumerate(target_text.split()):
          if t<len(target_text.split())-1:
              decoder_input_val_data[i, t] = target_token_index[word] # decoder input seq
          if t>0:
              # decoder target sequence (one hot encoded)
              # does not include the START_ token
              # Offset by one timestep
              decoder_target_val_data[i, t - 1, target_token_index[word]] = 1
'''

In [197]:
#encoder_input_val_data.shape

(278, 22)

In [201]:
#decoder_target_val_data.shape


(278, 27, 2969)

In [320]:
#decoder_input_data[0].shape
encoder_input_data.shape

(2773, 22)

# ENCODER DECODER MODEL ARCHITECTURE

In [55]:
#ENCODER DECODER MODEL ARCHITECTURE
latent_dim = 256

In [56]:
# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [57]:
#Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], 
              s)

In [58]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

In [59]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 256)    599552      input_2[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 256)    760064      input_3[0][0]                    
_______________________________________________________________________________________

In [253]:
#displaying model architecture
#from IPython.display import Image
#Image(retina=True, filename='train_model.png')

In [327]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 64
epochs = 200

In [255]:
y_test.shape

(278,)

# Training

In [329]:
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
           validation_split=0.2)

Epoch 1/200
35/35 [==============================] - 2s 46ms/step - loss: 1.4512 - acc: 0.1812 - val_loss: 1.4017 - val_acc: 0.1841
Epoch 2/200
35/35 [==============================] - 2s 45ms/step - loss: 1.4022 - acc: 0.1911 - val_loss: 1.3647 - val_acc: 0.1901
Epoch 3/200
35/35 [==============================] - 2s 46ms/step - loss: 1.3569 - acc: 0.2034 - val_loss: 1.3312 - val_acc: 0.2074
Epoch 4/200
35/35 [==============================] - 2s 47ms/step - loss: 1.3149 - acc: 0.2162 - val_loss: 1.3029 - val_acc: 0.2153
Epoch 5/200
35/35 [==============================] - 2s 46ms/step - loss: 1.2738 - acc: 0.2286 - val_loss: 1.2692 - val_acc: 0.2302
Epoch 6/200
35/35 [==============================] - 2s 45ms/step - loss: 1.2339 - acc: 0.2387 - val_loss: 1.2469 - val_acc: 0.2432
Epoch 7/200
35/35 [==============================] - 2s 45ms/step - loss: 1.1963 - acc: 0.2530 - val_loss: 1.2071 - val_acc: 0.2549
Epoch 8/200
35/35 [==============================] - 2s 47ms/step - loss: 1.

In [331]:
model.save_weights('/content/drive/My Drive/Colab Notebooks/LanguageTranslation/nmt_weights.h5') #saving model weight
model.save('/content/drive/My Drive/Colab Notebooks/LanguageTranslation/model.h5') #saving model

# Inference setup

In [332]:
#inference setup
# Encode the input sequence to get the "thought vectors"
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

# Function for decoding 

In [333]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    #print('sequences',input_seq)
    states_value = encoder_model.predict(input_seq)

    #print('states_values',states_value)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]

        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
        len(decoded_sentence) > 50):
              stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [174]:
#THIS GENERATOR FUNCTIO CAN BE USE FOR DATA GENERATION IN CASE IF WE WANT TO USE FIT_GENERATOR INSTEAD OF FIT AND WE CAN USE TRAIN AND TEST DATA
def generate_batch(X = X_train, y = y_train, batch_size = 128):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)          


# Data Testing

In [257]:
#RESULT TRAINING AND TESTING DATA AND GENERATE DATA USING GENERATOR_BATCH
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1

In [258]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

Input English sentence: he reminded his wife to wake him up at am
Actual Hindi Translation:  उसने अपनी पत्नी को उसे सुबह सात बजे उठाने की बात का याद दिलाया। 
Predicted Hindi Translation:  उसने अपनी पत्नी को उसे सुबह सात बजे उठाने की बा


In [259]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

Input English sentence: im looking for a small suitcase
Actual Hindi Translation:  मैं एक छोटी अटैची को ढूँढ रहा हूँ। 
Predicted Hindi Translation:  मैं एक छोटी अटैची को ढूँढ रहा हूँ। 


In [260]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

Input English sentence: he usually comes home late
Actual Hindi Translation:  वह आमतौर पर घर देर से आता है। 
Predicted Hindi Translation:  वह आमतौर पर घर देर से आता है। 


In [263]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

Input English sentence: i cant walk any further
Actual Hindi Translation:  मैं और नहीं चल सकती। 
Predicted Hindi Translation:  मुझसे और नहीं चला जाएगा। 


In [264]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

Input English sentence: you shouldve come yesterday
Actual Hindi Translation:  तुम्हे कल आना चाहिए था। 
Predicted Hindi Translation:  तुम्हे कल आना चाहिए था। 


In [265]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

Input English sentence: my hobby is visiting old temples
Actual Hindi Translation:  मैं पुराने मंदिरों को देखने जाने का शौकीन हूँ। 
Predicted Hindi Translation:  मैं पुराने मंदिरों को देखने जाने का शौकीन हूँ। 


In [266]:
#TESTING ON TESTING DATA
train_gen = generate_batch(X_test, y_test, batch_size = 1)
k=-1


In [267]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Hindi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

Input English sentence: dont shout
Actual Hindi Translation:  चिल्लाओ मत 
Predicted Hindi Translation:  चिल्लाईए मत। 


In [268]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Hindi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

Input English sentence: we ordered pink but we received blue
Actual Hindi Translation:  हमने पिंक रंग वाले का ऑर्डर किया था पर हमे नीले रंग वाला मिला। 
Predicted Hindi Translation:  हमने मामले की देखभाल अच्छी खुशी से गए। 


In [269]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Hindi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

Input English sentence: no answer is also an answer
Actual Hindi Translation:  जवाब न देना भी एक तरह का जवाब होता है। 
Predicted Hindi Translation:  तुम्हारी उम्र दिन मुझे लिखा है वह सच है। 


In [270]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Hindi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

Input English sentence: this watch is broken
Actual Hindi Translation:  यह घड़ी टूटी हुई है। 
Predicted Hindi Translation:  इस दवाई को है। 


In [271]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Hindi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

Input English sentence: i want you to go to osaka at once
Actual Hindi Translation:  फ़ौरन ओसाका जाओ। 
Predicted Hindi Translation:  मैं एक बूढ़े आदमी की तलाश की सलाह वाला हूँ। 


In [272]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Hindi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

Input English sentence: i like the blue one how much does it cost
Actual Hindi Translation:  मुझे नीली वाली पसंद है कितने की है 
Predicted Hindi Translation:  मुझे नीला वाला पसंद है कितने का है 


In [273]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Hindi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

Input English sentence: i am grateful to them
Actual Hindi Translation:  मैं उनका आभारी हूँ। 
Predicted Hindi Translation:  मैं परेशान था कि तो। 


In [274]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Hindi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

Input English sentence: the road is parallel to the river
Actual Hindi Translation:  वह सड़क नदी के साथ साथ चलती है। 
Predicted Hindi Translation:  एक टोपी इनसान की संगीत के पर बहुत खुश करो। 


In [275]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Hindi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

Input English sentence: i still dont know
Actual Hindi Translation:  मुझे अभी भी नहीं पता। 
Predicted Hindi Translation:  मुझे नहीं पता। 


In [276]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Hindi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

Input English sentence: were here to protect you
Actual Hindi Translation:  हम आपकी रक्षा करने के लिए यहाँ आए हैं। 
Predicted Hindi Translation:  आपकी रक्षा करना हमारा काम है। 


In [277]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Hindi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

Input English sentence: he has a lot of money
Actual Hindi Translation:  वह बहुत पैसेवाला है। 
Predicted Hindi Translation:  उसके अभी बहुत शौक लगती है। 


In [278]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Hindi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

Input English sentence: no i didnt go
Actual Hindi Translation:  नहीं मैं नहीं गया था। 
Predicted Hindi Translation:  मुझे नहीं पता गया कि क्या 


In [279]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Hindi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

Input English sentence: we must pay the tax
Actual Hindi Translation:  हमें टैक्स भरना पड़ेगा। 
Predicted Hindi Translation:  हमे अच्छी तक पहुँच गए। 


In [280]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Hindi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

Input English sentence: the trees are green
Actual Hindi Translation:  पेड़ हरे हैं। 
Predicted Hindi Translation:  कृपया पेड़ हैं। 


In [281]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Hindi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

Input English sentence: youre kidding
Actual Hindi Translation:  मज़ाक कर रहे हो 
Predicted Hindi Translation:  मौज करना। 


In [282]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Hindi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

Input English sentence: wash your hands before meals
Actual Hindi Translation:  खाना खाने से पहले अपने हाथ धोओ। 
Predicted Hindi Translation:  मेरी हाथ चल हैं। 


# TESTING ON USER INPUT DATA

In [369]:
def translation():
    
    list_exit_word=['exit','break','leave','bye']
    
    while True:
        input_text=input('Enter the text you want to translate \t')
        if input_text not in list_exit_word:
            thought_matrix=convert_user_input_into_mat(input_text)
            response=decode_sequence(thought_matrix)
            response=response.replace('_END','') #replacing _END with space
            print('original text \t',input_text)
            print('translated text \t',response)
        else:
            print('Have good day')
            return 


def convert_user_input_into_mat(input_text): #this method is same as we did above with data before training
    input_text=input_text.lower()
    encoder_input_data = np.zeros((1, max_length_src),dtype='float32') #creating matrix of max_le

    for t, word in enumerate(input_text.split()):
      #print(word)
      encoder_input_data[0, t] = input_token_index[word] #input_token_index is dict and our vocabulary is limited to if we enter any word which is not present in our vocab we 
      #will get an error

    return encoder_input_data

In [372]:
translation()#AS WE CAN SEE THAT THE ACCURACY IS NOT SO GOOD BECAUSE OF 2 MAJOR REASONS FIRST ONE IS THE DATA IS VERY SMALL AND SECOND ONE IS BECAUSE OF SMALL DATASET 
#UNCERTAINITY IS HIGH IN THE PERFORMANCE WE CAN USE THE CHARACTER LEVEL TRANSLATION INSTEAD OF WORD LEVEL BUT IN MOST OF CASES WORD LEVEL TRASNLATION IS USED
#AND ONE ADVANTAGE OF CHARACTER LEVEL TRANSLATION OVER WORD LEVEL IS THAT IN WORD LEVEL WE HAVE LIMITED VOCAB SO IF ANY WORD IS NOT PRESENT IN VOCAB THEN WE WILL GET AN ERROR
#BUT IN CASE OF CHARACTERS AS WE KNOW CHARACTERS ARE LIMITED IN EVERY LANGUAGE WE WILL GET GOOD RESULT COMPARATIVE TO WORD LEVEL SPEACIALLY WHEN WE HAVE SMALL DATASET.

Enter the text you want to translate 	HOW ARE YOU
original text 	 HOW ARE YOU
translated text 	  तुम कैसी हो 
Enter the text you want to translate 	I AM GOOD
original text 	 I AM GOOD
translated text 	  मुझे अंग्रेज़ी अच्छा लगता है। 
Enter the text you want to translate 	WHAT IS YOUR AGE
original text 	 WHAT IS YOUR AGE
translated text 	  तुम्हारी उम्र क्या है 
Enter the text you want to translate 	i do not know
original text 	 i do not know
translated text 	  मुझे नहीं पता। 
Enter the text you want to translate 	ok
original text 	 ok
translated text 	  मैं समझता हूँ। 
Enter the text you want to translate 	take care
original text 	 take care
translated text 	  बचाओ 
Enter the text you want to translate 	hello
original text 	 hello
translated text 	  नमस्ते। 
Enter the text you want to translate 	good
original text 	 good
translated text 	  मज़े करना। 
Enter the text you want to translate 	exit
Have good day
